# Optimization with JuMP

Julia has several packages for optimization, root-finding, etc. 
For optimization, there are pure-Julia routines in Optim.jl, and an interface to the NLOpt package in NLOpt.jl. 

But one of the crown jewels of Julia is JuMP.jl (which stands for Julia Mathematical Programming, i.e. mathematical optimization). This provides a "domain-specific language" (DSL) for specifying constrained optimization problems, which it converts into the correct syntax for a range of commercial and open-source optimization packages ("solvers").

Let's see a couple of examples, taken from the JuMP documentation. First a linear programming example:

In [ ]:
Pkg.add("JuMP")
Pkg.add("Clp")  # need to add a solver

In [ ]:
using JuMP
using Clp

m = Model(solver = ClpSolver())
@variable(m, 0 <= x <= 2 )
@variable(m, 0 <= y <= 30 )

@objective(m, Max, 5x + 3*y )
@constraint(m, 1x + 5y <= 3.0 )

print(m)

status = solve(m)

println("Objective value: ", getobjectivevalue(m))
println("x = ", getvalue(x))
println("y = ", getvalue(y))

## Maximum likelihood

In [ ]:
Pkg.add("Ipopt")

In [ ]:
Pkg.build

In [ ]:
#  Copyright 2017, Iain Dunning, Joey Huchette, Miles Lubin, and contributors
#  This Source Code Form is subject to the terms of the Mozilla Public
#  License, v. 2.0. If a copy of the MPL was not distributed with this
#  file, You can obtain one at http://mozilla.org/MPL/2.0/.
using JuMP, Ipopt

# Use nonlinear optimization to compute the maximum likelihood estimate (MLE)
# of the parameters of a normal distribution
# aka the sample mean and variance

n = 1000
data = randn(n)

m = Model(solver=IpoptSolver(print_level=0))

@variable(m, μ, start = 0.0)
@variable(m, σ >= 0.0, start = 1.0)

@NLobjective(m, Max, (n/2)*log(1/(2π*σ^2))-sum((data[i]-μ)^2 for i=1:n)/(2σ^2))

solve(m)

println("μ = ", getvalue(μ))
println("mean(data) = ", mean(data))
println("σ^2 = ", getvalue(σ)^2)
println("var(data) = ", var(data))
println("MLE objective: ", getobjectivevalue(m))

# constrained MLE?
@NLconstraint(m, μ == σ^2)

solve(m)
println("\nWith constraint μ == σ^2:")
println("μ = ", getvalue(μ))
println("σ^2 = ", getvalue(σ)^2)

println("Constrained MLE objective: ", getobjectivevalue(m))